In [ ]:
!pip install confluent-kafka

In [ ]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4

In [ ]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [ ]:
conf = {'bootstrap.servers': bootstrap_server}

In [ ]:
a = AdminClient(conf)

In [ ]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    ##if not (str(t)).startswith("_"):
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

In [ ]:
#fs = a.create_topics([NewTopic("test1p", num_partitions=1, replication_factor=1)])

#for topic, f in fs.items():
#    try:
#        f.result()  # The result itself is None
#        print("Topic {} created".format(topic))
#    except Exception as e:
#        print("Failed to create topic {}: {}".format(topic, e))

In [ ]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *
import time

topic = "TemperatureSensorEvent"

def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [ ]:
producer_conf = {
        'bootstrap.servers': bootstrap_server,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': StringSerializer('utf_8')
}

producer = SerializingProducer(producer_conf)

## run the following cell to demostrate that fire is not detected

In [ ]:
import json
from random import gauss

while True:
    key = "S1"
    value = {"sensor": "S1","temperature": gauss(20, 1.0),"ts":int(time.time())}
    producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)


## run the following cell to demostrate to detect fire

In [ ]:
while True:
    key = "S1"
    value = {"sensor": "S1","temperature": gauss(55, 1.0),"ts":int(time.time())}
    producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)